In [1]:
import tensorflow as tf
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Dropout,GlobalAveragePooling2D
from keras.applications.mobilenet import MobileNet, preprocess_input
import math
import numpy as np

In [2]:
TRAIN_DATA_DIR = r"C:\Users\TANISHKA\OneDrive\Desktop\Train2"
VALIDATION_DATA_DIR = r"C:\Users\TANISHKA\OneDrive\Desktop\Validation"
TRAIN_SAMPLES = 500
VALIDATION_SAMPLES = 250
NUM_CLASSES = 5
IMG_WIDTH, IMG_HEIGHT = 224, 224
BATCH_SIZE = 64


In [3]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
rotation_range=20,
width_shift_range=0.2,
height_shift_range=0.2,
zoom_range=0.2)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [4]:
train_generator = train_datagen.flow_from_directory(
TRAIN_DATA_DIR,
target_size=(IMG_WIDTH, IMG_HEIGHT),
batch_size=BATCH_SIZE,
shuffle=True,
seed=12345,
class_mode='categorical')
validation_generator = val_datagen.flow_from_directory(
VALIDATION_DATA_DIR,
target_size=(IMG_WIDTH, IMG_HEIGHT),
batch_size=BATCH_SIZE,
shuffle=False,
class_mode='categorical')

Found 500 images belonging to 5 classes.
Found 250 images belonging to 5 classes.


In [5]:
def model_maker():
    base_model = MobileNet(include_top=False, input_shape =(IMG_WIDTH,IMG_HEIGHT,3))
    for layer in base_model.layers[:]:
        layer.trainable = False # Freeze the layers
    input = Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    custom_model = base_model(input)
    custom_model = GlobalAveragePooling2D()(custom_model)
    custom_model = Dense(64, activation='relu')(custom_model)
    custom_model = Dropout(0.5)(custom_model)
    predictions = Dense(NUM_CLASSES, activation='softmax')(custom_model)
    return Model(inputs=input, outputs=predictions)

In [6]:
model = model_maker()
model.compile(loss='categorical_crossentropy',
optimizer= tf.optimizers.Adam(learning_rate=0.001),
metrics=['acc'])
num_steps = math.ceil(float(TRAIN_SAMPLES)/BATCH_SIZE)
model.fit(train_generator,
steps_per_epoch = num_steps,
epochs=10,
validation_data = validation_generator,
validation_steps = num_steps)



Epoch 1/10


8/8 [==============================] - 40s 4s/step - loss: 1.7000 - acc: 0.3520 - val_loss: 1.0461 - val_acc: 0.6360
Epoch 2/10
8/8 [==============================] - 21s 3s/step - loss: 1.1579 - acc: 0.5340
Epoch 3/10
8/8 [==============================] - 21s 3s/step - loss: 0.8509 - acc: 0.6960
Epoch 4/10
8/8 [==============================] - 21s 3s/step - loss: 0.7238 - acc: 0.7440
Epoch 5/10
8/8 [==============================] - 22s 3s/step - loss: 0.6377 - acc: 0.7620
Epoch 6/10
8/8 [==============================] - 21s 3s/step - loss: 0.5884 - acc: 0.7800
Epoch 7/10
8/8 [==============================] - 14s 2s/step - loss: 0.5387 - acc: 0.8100
Epoch 8/10
8/8 [==============================] - 13s 2s/step - loss: 0.4867 - acc: 0.8280
Epoch 9/10
8/8 [==============================] - 13s 2s/step - loss: 0.4606 - acc: 0.8380
Epoch 10/10
8/8 [==============================] - 13s 2s/step - loss: 0.3993 - acc: 0.8440


In [11]:
model.save('Ailment.keras')


In [12]:
from keras.models import load_model
model = load_model('Ailment.keras')

In [13]:
img_path = r"C:\Users\TANISHKA\OneDrive\Desktop\Validation\ACNE\rosacea-nose-52.jpg"
img = image.load_img(img_path, target_size=(224,224))
img_array = image.img_to_array(img)
expanded_img_array = np.expand_dims(img_array, axis=0)
preprocessed_img = preprocess_input(expanded_img_array) # Preprocess the image
prediction = model.predict(preprocessed_img)
highest=prediction.argmax()
result=list(validation_generator.class_indices.keys())[list(validation_generator.class_indices.values()).index(highest)]
print("SORRY TO SAY BUT AAPKO {} HUA HAI!!".format(result))




1/1 [==============================] - 1s 1s/step
SORRY TO SAY BUT AAPKO ACNE HUA HAI!!


In [14]:
print(validation_generator.class_indices)

{'ACNE': 0, 'ECZEMA': 1, 'KELOIDALIS_ACNE_(YOU_WILL_SUFFER_FROM_HAIRLOSS)': 2, 'MELANOMA_SKIN_CANCER': 3, 'NAIL_FUNGUS': 4}
